In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mdtraj as md

%matplotlib inline

In [ ]:
def get_sequence(work_dir,prmtop_name,nc_file):
        
    traj = md.load_netcdf(work_dir+nc_file, top=work_dir+prmtop_name, stride = None)

    res_list = []
    for res in traj.topology.residues:
        res_list.append(res)

    res_list_strings = []

    for res in res_list:
        res_list_strings.append(str(res))
    
    return res_list_strings

In [ ]:
def helicity_calculator_from_dat(dat_file, max_frame):
    from collections import defaultdict

    ss_data = defaultdict(list)  # resid -> list of structure states across frames

    with open(dat_file, 'r') as f:
        for line in f:
            if line.startswith("#") or not line.strip():
                continue  # skip comments and blanks

            parts = line.strip().split()
            if len(parts) != 5:
                continue  

            resid = int(parts[0])
            frame = int(parts[3])
            ss = parts[4]

            if frame <= max_frame:
                ss_data[resid].append(ss)

    helicity_per_residue = []

    for resid in sorted(ss_data.keys()):
        states = ss_data[resid]
        total = len(states)
        if total == 0:
            helicity_per_residue.append(0)
            continue

        helix_codes = {'H', 'G', 'I'}
        h_count = sum(1 for s in states if s.strip().upper() in helix_codes)
        percent_helicity = (h_count / total) * 100
        helicity_per_residue.append(percent_helicity)

    return helicity_per_residue


In [ ]:
def auc_calculator(helicity):
    auc = np.trapz(helicity, dx=1)

    return auc

In [ ]:
# create reference helicity plot for 100% helicity (excluding C-terminus)

reference_helicity = [100.0,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 0.0]

ref_auc = np.trapz(reference_helicity, dx=1)
ref_auc

In [ ]:
work_dirs = ['./', './replicate-2/', './replicate-3/']

In [ ]:
prmtop_name = 'dry.prmtop'
nc_file = "prod_dry.nc"

In [ ]:
for work_dir in work_dirs:

    dat_file = work_dir + "helicity.tml"  

    helicity_list = helicity_calculator_from_dat(dat_file,2001)
    auc = auc_calculator(helicity_list)
    sequence = get_sequence(work_dir=work_dir,prmtop_name=prmtop_name,nc_file=nc_file)

    fig,ax = plt.subplots()
    ax.plot(sequence[1:-1], helicity_list) 

    plt.xticks(rotation=85)

    ax.set_ylim([0.0, 105.0])
    plt.gcf().set_size_inches(15,5)

    plt.show()

In [ ]:
replicates_helicity = []
replicates_auc = []

for work_dir in work_dirs:
    print(work_dir)

    dat_file = work_dir + "helicity.tml"  

    increments = range(10, 2001, 10)
    all_helicity_results = []

    for inc in increments:
        helicity = helicity_calculator_from_dat(dat_file, inc)
        all_helicity_results.append(helicity)


    auc_list = []

    for hel in all_helicity_results:
        auc = auc_calculator(hel)

        auc_list.append(auc)
    
    replicates_helicity.append(all_helicity_results)
    replicates_auc.append(auc_list)

In [ ]:
replicates_percent_helicity = []

for replica_auc in replicates_auc:
    percent_helicity = []

    for auc in replica_auc:
        percent_helicity.append((auc/ref_auc)*100)

    replicates_percent_helicity.append(percent_helicity)

In [ ]:
# one frame is 0.01 ns

time = range(1, 201, 1) 


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(time, replicates_auc[0], marker='', linestyle='-', color='b', markersize=3, label="replicate 1")
plt.plot(time, replicates_auc[1], marker='', linestyle='-', color='r', markersize=3, label="replicate 2")
plt.plot(time, replicates_auc[2], marker='', linestyle='-', color='g', markersize=3, label="replicate 3")

# Labels and title
plt.xlabel("time / ns")
plt.ylabel("AUC")

plt.legend()
#ßplt.grid(True)
plt.xlim([0.0, 200.0])
plt.ylim([0.0, 1400.0])

plt.rcParams['figure.dpi'] = 300
fig.savefig(work_dirs[0]+'auc.png', bbox_inches='tight')
plt.show()


In [ ]:

plt.figure(figsize=(10, 5))
plt.plot(time, replicates_percent_helicity[0], marker='', linestyle='-', color='b', markersize=3, label="replicate 1")
plt.plot(time, replicates_percent_helicity[1], marker='', linestyle='-', color='r', markersize=3, label="replicate 2")
plt.plot(time, replicates_percent_helicity[2], marker='', linestyle='-', color='g', markersize=3, label="replicate 3")

# Labels and title
plt.xlabel("time / ns")
plt.ylabel("% Helicity")

plt.legend()
#plt.grid(True)
plt.xlim([0.0, 200.0])
plt.ylim([0.0, 100.0])

plt.rcParams['figure.dpi'] = 300
fig.savefig(work_dirs[0]+'helicity.png', bbox_inches='tight')
plt.show()


In [ ]:
data = np.array([replicates_percent_helicity[0], replicates_percent_helicity[1], replicates_percent_helicity[2]])  

average_list = np.mean(data, axis=0)

std_err_list = np.std(data, axis=0)/np.sqrt(len(work_dirs))

In [ ]:
plt.figure(figsize=(10, 5))

plt.errorbar(time, average_list, yerr=std_err_list, fmt='', color='violet', alpha=0.5, capsize=2)
plt.plot(time, average_list, color="purple", linewidth=2)

# Labels and title
plt.xlabel("time / ns", fontsize=35)
plt.ylabel("% Helicity", fontsize=35)

plt.xticks(fontsize=25)
plt.yticks(fontsize=25)

# plt.legend()
#plt.grid(True)
plt.xlim([0.0, 200.0])
plt.ylim([0.0, 100.0])

plt.rcParams['figure.dpi'] = 300
fig.savefig(work_dirs[0]+'avg_helicity.png', bbox_inches='tight')
plt.show()

In [ ]:
print(average_list[-1], std_err_list[-1])